# Generative Adversarial Network to generate images 

## A bunch of imports 

In [ ]:
from keras import backend as K
K.set_image_dim_ordering('th') # ensure our dimension notation matches
import numpy as np
import time
#from tensorflow.examples.tutorials.mnist import input_data

from keras.models import Sequential 
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D,Convolution2D, Conv2DTranspose, UpSampling2D, MaxPool2D
from keras.layers import LeakyReLU, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop, SGD
from keras.layers.convolutional import AveragePooling2D
from keras.datasets import mnist
import matplotlib.pyplot as pplot
from PIL import Image


## The generator method
the method generates images from random data 

In [ ]:
def getGenerator():
    
    model = Sequential()
    model.add(Dense(input_dim=100, output_dim=1024))
    model.add(Activation('tanh'))
    model.add(Dense(128*7*7))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Reshape((128, 7, 7), input_shape=(128*7*7,)))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Convolution2D(64, 5, 5, border_mode='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Convolution2D(1, 5, 5, border_mode='same'))
    model.add(Activation('tanh'))
    return model

## The discriminator method
the method predicts whether input image is real or fake 

In [ ]:
def getDiscriminator():
    model = Sequential()
    model.add(Convolution2D(
                        64, 5, 5,
                        border_mode='same',
                        input_shape=(1, 28, 28)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(128, 5, 5))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model
    

## Adversarial network


In [ ]:
def getAdversarial(generator, discriminator):    
    
    AM = Sequential()
    AM.add(generator)
    discriminator.trainable = False;
    AM.add(discriminator)
    return AM

## Image utility methods 
combine_image is used to merge images into one image for display purposes

In [ ]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[2:]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[0, :, :]
    return image

def SaveImage(image,fileName):
     image = image*127.5+127.5
     pplot.imshow(Image.fromarray(image.astype(np.uint8)))
     Image.fromarray(image.astype(np.uint8)).save(fileName+".png")

## train method.. 

In [ ]:
def train(generator, discriminator,adversarial, epochs=100,batch_size=1000):
    
    save_interval = 0
    k = 1       # as in the GAN paper
    
    noise_input = None
    if save_interval>0:
        noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
    for e in range(epochs):     # epochs
        for indBatch in range(int(x_train.shape[0]/batch_size)):
            for j in range(0,k):        # loop for trianing of the discriminator
                #images_train = x_train[np.random.randint(0,x_train.shape[0], size=batch_size), :, :, :]
                images_train = x_train[indBatch*batch_size:(indBatch+1)*batch_size, :, :, :]
                noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
                images_fake = generator.predict(noise)
                if indBatch % 20 == 0:
                    image = combine_images(images_fake)
                    image = image*127.5+127.5
                    Image.fromarray(image.astype(np.uint8)).save(
                            "imagesGAN01\\"+str(e)+"_"+str(indBatch)+".png")
                x = np.concatenate((images_train, images_fake))
                y = np.ones([2*batch_size, 1])
                y[batch_size:, :] = 0
                discriminator.trainable = True;
                d_loss = discriminator.train_on_batch(x, y)
            y = np.ones([batch_size, 1])
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100]) # sample minibatch of noise
            discriminator.trainable = False;
            a_loss = adversarial.train_on_batch(noise, y)
            discriminator.trainable = True;
            
            log_mesg = "e = %d: %d [D loss: %f, acc: %f]" % (e, indBatch ,d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            
            print(log_mesg)
            if save_interval>0:
                if (e+1)%save_interval==0:
                    plot_images(save2file=True, samples=noise_input.shape[0],\
                        noise=noise_input, step=(e+1))

## we are prepared now. lets start the main task now... 

In [ ]:
img_rows = 28
img_cols = 28
batch_size = 1000

#x_train = input_data.read_data_sets("mnist",\
#                one_hot=True).train.images
#x_train = x_train.reshape(-1, img_rows,\
#                img_cols, 1).astype(np.float32)
#x_train = (x_train.astype(np.float32) - 127.5)/127.5


(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train.astype(np.float32) - 127.5)/127.5
x_train = x_train.reshape((x_train.shape[0], 1) + x_train.shape[1:])


generator = getGenerator()
discriminator = getDiscriminator()
adversarial = getAdversarial(generator,discriminator)

adversarial.summary()
d_optimiser = SGD(lr=0.0005, momentum=0.9, nesterov=True)
g_optimiser = SGD(lr=0.0005, momentum=0.9, nesterov=True)

#optimizer = RMSprop(lr=0.0004, clipvalue=1.0, decay=3e-8)
generator.compile(loss='binary_crossentropy',optimizer = "SGD",metrics=['accuracy'])
adversarial.compile(loss='binary_crossentropy',optimizer = g_optimiser,metrics=['accuracy'])
discriminator.trainable = True;
discriminator.compile(loss='binary_crossentropy',optimizer = d_optimiser,metrics=['accuracy'])

train(generator, discriminator,adversarial)

#AM.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


noise = np.random.uniform(-1.0, 1.0, size=[16, 100])
images_fake = generator.predict(noise)
#
for i in range(images_fake.shape[0]):
    pplot.subplot(4, 4, i+1)
    image = images_fake[i, :, :, :]
    image = np.reshape(image, [img_rows, img_cols])
    pplot.imshow(image, cmap='gray')
    pplot.axis('off')
pplot.show()
